<a href="https://colab.research.google.com/github/Roni-hub18/BraunTumor.Project/blob/main/braintumor1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
from scipy.stats import kurtosis, skew
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pywt
import pandas as pd
from sklearn.decomposition import PCA
import warnings

warnings.filterwarnings("ignore")

# Function to resize and convert image to grayscale
def resize_and_convert_to_gray(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to load image at {image_path}. Skipping...")
        return None
    resized_image = cv2.resize(image, (512, 512))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    return grayscale_image

# Adaptive mean thresholding function
def adaptive_mean_thresholding(image):
    return cv2.adaptiveThreshold(
        image, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY, blockSize=11, C=2
    )

# Histogram-based (Otsu's) thresholding function
def otsu_thresholding(image):
    _, thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresholded_image

# Function to combine adaptive mean and Otsu thresholding using weighted addition
def combine_thresholding(adaptive_thresh_image, otsu_thresh_image):
    combined_image = cv2.addWeighted(adaptive_thresh_image, 0.5, otsu_thresh_image, 0.5, 0)
    return combined_image

# K-means segmentation
def kmeans_segmentation(image, num_clusters=2, max_iter=100, tol=1e-4):
    flattened_image = image.flatten().reshape(-1, 1)
    kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=42, max_iter=max_iter, tol=tol)
    kmeans.fit(flattened_image)
    cluster_labels = kmeans.labels_.reshape(image.shape)
    return cluster_labels

# Function to segment clusters based on K-means labels
def segment_clusters(image, cluster_labels):
    segmented_image = np.zeros_like(image)
    if np.mean(image[cluster_labels == 0]) > np.mean(image[cluster_labels == 1]):
        segmented_image[cluster_labels == 0] = 255
    else:
        segmented_image[cluster_labels == 1] = 255
    return segmented_image

# Feature extraction using GLCM, Wavelet, and statistical measures
def extract_features(image):
    coeffs = pywt.dwt2(image, 'haar')
    cA, (cH, cV, cD) = coeffs

    glcm = graycomatrix(image, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

    contrast = graycoprops(glcm, 'contrast').mean()
    correlation = graycoprops(glcm, 'correlation').mean()
    energy = graycoprops(glcm, 'energy').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()

    mean = np.mean(cA)
    std_dev = np.std(cA)
    entropy_val = shannon_entropy(cA)
    kurtosis_val = kurtosis(cA.flatten())
    skewness_val = skew(cA.flatten())

    features = [
        contrast, correlation, energy, homogeneity,
        mean, std_dev, entropy_val, kurtosis_val, skewness_val
    ]
    return features

# Function to predict from a folder of images
def predict_from_folder(folder_path, svm_classifier, pca):
    predictions = []
    image_names = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            grayscale_image = resize_and_convert_to_gray(image_path)

            if grayscale_image is None:
                continue

            # Apply adaptive mean thresholding
            adaptive_thresh_image = adaptive_mean_thresholding(grayscale_image)

            # Apply Otsu's thresholding
            otsu_thresh_image = otsu_thresholding(grayscale_image)

            # Combine the results of adaptive and Otsu's thresholding
            combined_image = combine_thresholding(adaptive_thresh_image, otsu_thresh_image)

            # Apply K-means segmentation
            cluster_labels = kmeans_segmentation(combined_image)

            # Segment clusters
            segmented_image = segment_clusters(grayscale_image, cluster_labels)

            # Extract features
            features = extract_features(segmented_image)

            # Replace NaN values with 0
            features = np.nan_to_num(features)

            # Apply PCA transformation
            features_pca = pca.transform([features])

            # Predict using SVM classifier
            prediction = svm_classifier.predict(features_pca)[0]
            predictions.append(prediction)
            image_names.append(filename)
            print(f"Image: {filename} --> Prediction: {prediction}")

    # Return predictions as a dataframe
    results_df = pd.DataFrame({
        'Image_Name': image_names,
        'Prediction': predictions
    })
    return results_df

# Main processing
input_folders = {
    'no': r'/content/drive/My Drive/Brain_Tumor_Detection/no',
    'yes': r'/content/drive/My Drive/Brain_Tumor_Detection/yes'
}

all_features = []
all_labels = []

# Extract features and labels from images in folders
for label, folder_path in input_folders.items():
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            grayscale_image = resize_and_convert_to_gray(image_path)

            if grayscale_image is None:
                continue

            # Apply adaptive mean thresholding
            adaptive_thresh_image = adaptive_mean_thresholding(grayscale_image)

            # Apply Otsu's thresholding
            otsu_thresh_image = otsu_thresholding(grayscale_image)

            # Combine the results
            combined_image = combine_thresholding(adaptive_thresh_image, otsu_thresh_image)

            # Apply K-means segmentation
            cluster_labels = kmeans_segmentation(combined_image)

            # Segment clusters
            segmented_image = segment_clusters(grayscale_image, cluster_labels)

            # Extract features
            features = extract_features(segmented_image)

            # Replace NaN values with 0
            features = np.nan_to_num(features)

            all_features.append(features)
            all_labels.append(label)

# Convert feature lists to numpy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=min(X.shape[0], X.shape[1]))
X_pca = pca.fit_transform(X)

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_imputed_pca = imputer.fit_transform(X_pca)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed_pca, y, test_size=0.2, random_state=42, stratify=y)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear', probability=True, random_state=42)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f"\nModel Accuracy with PCA: {accuracy * 100:.2f}%")
print("\nClassification Report with PCA:\n", report)
print("Confusion Matrix with PCA:\n", conf_matrix)

# Make predictions on a new folder of images
prediction_folder = '/content/drive/My Drive/Brain_Tumor_Detection/yes'
results = predict_from_folder(prediction_folder, svm_classifier, pca)

# Print prediction results
print("\nPrediction Results with PCA:")
print(results)


Model Accuracy with PCA: 72.17%

Classification Report with PCA:
               precision    recall  f1-score   support

          no       0.73      0.70      0.72       300
         yes       0.71      0.74      0.73       300

    accuracy                           0.72       600
   macro avg       0.72      0.72      0.72       600
weighted avg       0.72      0.72      0.72       600

Confusion Matrix with PCA:
 [[210  90]
 [ 77 223]]
Image: y1435.jpg --> Prediction: yes
Image: y1444.jpg --> Prediction: no
Image: y1440.jpg --> Prediction: yes
Image: y1452.jpg --> Prediction: yes
Image: y1453.jpg --> Prediction: yes
Image: y1458.jpg --> Prediction: yes
Image: y1461.jpg --> Prediction: yes
Image: y1454.jpg --> Prediction: yes
Image: y1451.jpg --> Prediction: no
Image: y1457.jpg --> Prediction: yes
Image: y1456.jpg --> Prediction: yes
Image: y1455.jpg --> Prediction: no
Image: y1450.jpg --> Prediction: no
Image: y1460.jpg --> Prediction: yes
Image: y146.jpg --> Prediction: yes
Image

In [ ]:
pip install os-sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 28.0 MB/s eta 0:00:00
